In [22]:
import numpy as np
import scipy.io as spio
import scipy.stats as stats
import math


class Node:
    def __init__(self,feature,threshold,left,right,label):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.label=label

def calculate_max_label_count(labels):
    count_map={}
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
            
     ##check this       
    return max(count_map.values())
        
def calculate_max_label(labels):
    count_map={}
    curr_max=0
    max_label=0
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
        if(count_map[labels[i]]>curr_max):
            curr_max=count_map[labels[i]]
            max_label=labels[i]
     ##check this       
    return max_label

def entropy(labels):
    if labels is None or len(labels)==0:
        return 0;
    return (len(labels)-calculate_max_label_count(labels))*1.0*(1.0/len(labels))
    

def calculate_optimal_feature_threshold(feature_space,labels):
    opt_f=0
    opt_t=0
    max_uncert_red=0.0
    
    for f in range(feature_space.shape[1]):
        a=(max(feature_space[:,f])-min(feature_space[:,f]))/10
        b=[min(feature_space[:,f])+i*a for i in range(10)]
        for t in b:
            left_labels=[]
            right_labels=[]
            for i in range(len(feature_space)):
                if(feature_space[i,f]<t):
                    left_labels.append(labels[i])
                else:
                    right_labels.append(labels[i])
            if(len(left_labels)==0 or len(right_labels)==0):
                continue
            uncertainty_red = entropy(labels)-(len(left_labels)*1.0/len(labels))*entropy(left_labels) - (len(left_labels)*1.0/len(labels))*entropy(right_labels)
            if(uncertainty_red > max_uncert_red):
                opt_f=f
                opt_t=t
    
    return [opt_f,opt_t]

def build_tree(feature_space,labels,param):
    
    if len(feature_space)<=param:
        return Node(None,None,None,None,calculate_max_label(labels))
    left_space=[]
    left_labels=[]
    right_space=[]
    right_labels=[]
    
    [f,t]=calculate_optimal_feature_threshold(feature_space,labels)
    print(f,t)
    if(f==0 and t== 0):
        return Node(None,None,None,None,calculate_max_label(labels))
    
    for i in range(len(feature_space)):
        if(feature_space[i][f]<t):
            
            left_space.append(feature_space[i])
            left_labels.append(labels[i])
        else:
            right_space.append(feature_space[i])
            right_labels.append(labels[i])
    
    left_node=build_tree(np.array(left_space),left_labels,param)
    right_node=build_tree(np.array(right_space),right_labels,param)
    
    return Node(f,t,left_node,right_node,None)
    
    ## find the optimal feature and threshold
    ## split on the optimal feature and threshold
    ## node.left=build_tree(left_space)
    ## node.right=build_tree(right_space)
    ## return node


class DecisionTree:
    
    def __init__(self,num_classes,root):
        self.num_classes=num_classes
        self.root=root

                

    def train_model(self,training_data,labels,param):
        print("------Pre-processing Data-----")
        for i in range(len(training_data)):
            if (i == 0):
                feature_mle_mu = training_data[i]
                self.dim = (training_data[i].shape)[0]
            else:
                feature_mle_mu += training_data[i]
            
        feature_mle_mu = feature_mle_mu * 1.0/len(training_data)
        
        for i in range(len(training_data)):
            x_minus_mu_feature = training_data[i]-feature_mle_mu
            if (i == 0):
                feature_mle_sigma = x_minus_mu_feature**2
            else:
                feature_mle_sigma += x_minus_mu_feature**2
                
        feature_mle_sigma = feature_mle_sigma * 1.0/len(training_data)
        
        keys = np.linspace(0,self.dim-1,self.dim)
        feature_var = list(zip(keys,feature_mle_sigma))
        
        feature_var_sorted = sorted(feature_var, key=lambda f: f[1], reverse = True)
        self.top200 = [int(e[0]) for e in feature_var_sorted[:200]]  
        print("------Done Pre-processing Data -----")
        # slice the data
        training_data = training_data[:,self.top200]

        self.root=build_tree(training_data,labels,param)

    def predict(self,x):
        prediction=None
        curr_node=self.root
        while prediction==None:
            if curr_node.label!= None:
                prediction=curr_node.label
            else:
                if(x[curr_node.feature]<curr_node.threshold):
                    curr_node=curr_node.left
                else:
                    curr_node=curr_node.right
        return prediction
                
                
                
                
                
        
    def testing_error(self,test_data,labels):
        misses=0
        test_data = test_data[:,self.top200]
        for i in range(len(test_data)):
            if(self.predict(test_data[i])!=labels[i]):
                misses=misses+1
        return (misses*1.0/len(test_data))
            
    
        ##Steps:
        ##1: return argmax P[X=x/Y=y]*P[Y=y]

mat = spio.loadmat('hw1data.mat', squeeze_me=True)
image_matrix=np.asarray(mat['X'], dtype=np.int32)

label_array=np.asarray(mat['Y'], dtype=np.int32)

decision_tree= DecisionTree(10,None)
#[f,t]= calculate_optimal_feature_threshold(image_matrix[:30],label_array[:30])  
decision_tree.train_model(image_matrix[0:1000,],label_array[0:1000],10)
print(decision_tree.testing_error(image_matrix[1000:1200,],label_array[1000:1200]))




------Pre-processing Data-----
------Done Pre-processing Data -----
199 60604.200000000004
144 25.5
186 25.5
191 25.5
199 102.0
182 72.0
182 40.5
174 25.4
191 2.6999999999999997
186 22.5
185 228.6
184 150.60000000000002
182 18.0
188 51.0
198 227.70000000000002
195 228.6
195 194.4
195 171.0
195 149.4
195 117.0
195 95.39999999999999
195 13.6
194 220.5
194 43.199999999999996
193 168.29999999999998
193 115.2
195 9.0
193 51.300000000000004
190 228.6
187 228.6
187 122.39999999999999
187 97.2
187 83.7
199 77.39999999999999
192 80.10000000000001
196 227.70000000000002
192 25.2
182 12.6
141 102.60000000000001
0 0
198 101.6
199 89.10000000000001
199 41.4
199 32.4
199 28.8
199 6.3
199 3.6
197 209.70000000000002
197 181.79999999999998
197 144.9
198 209.70000000000002
197 36.9
198 158.4
198 138.6
197 23.400000000000002
198 113.39999999999999
198 85.5
197 3.6
190 207.9
190 175.5
198 58.5
198 9.9
195 228.6
198 8.1
195 201.6
195 178.20000000000002
194 1.8
186 22.5
186 17.099999999999998
193 177.1
175 

In [24]:
decision_tree.train_model(image_matrix[0:1000,],label_array[0:1000],1)

print(decision_tree.testing_error(image_matrix[0:1000,],label_array[0:1000]))
#print(decision_tree.testing_error(image_matrix[:300,],label_array[:300]))


------Pre-processing Data-----
------Done Pre-processing Data -----
199 121208.40000000001
144 25.5
186 25.5
191 25.5
199 102.0
182 72.0
182 40.5
174 25.4
191 2.6999999999999997
186 22.5
185 228.6
184 150.60000000000002
182 18.0
188 51.0
198 227.70000000000002
195 228.6
195 194.4
195 171.0
195 149.4
195 117.0
195 95.39999999999999
195 13.6
194 220.5
194 43.199999999999996
193 168.29999999999998
193 115.2
195 9.0
193 51.300000000000004
190 228.6
187 228.6
187 122.39999999999999
187 97.2
187 83.7
199 77.39999999999999
192 80.10000000000001
196 227.70000000000002
192 25.2
182 12.6
141 102.60000000000001
0 0
0 0
0 0
0 0
198 101.6
199 37.800000000000004
0 0
198 240.0
198 212.10000000000002
199 20.7
199 22.9
198 251.8
0 0
0 0
199 89.10000000000001
199 41.4
199 32.4
199 28.8
199 6.3
199 3.6
197 209.70000000000002
197 181.79999999999998
197 144.9
198 209.70000000000002
197 36.9
198 158.4
198 138.6
197 23.400000000000002
198 113.39999999999999
198 85.5
197 3.6
190 207.9
190 175.5
198 58.5
198 9

199 172.79999999999998
199 96.3
199 57.6
197 13.5
196 26.099999999999998
195 228.6
198 56.699999999999996
195 115.2
195 43.199999999999996
195 17.099999999999998
194 107.10000000000001
194 75.60000000000001
193 228.6
191 217.7
191 188.9
0 0
195 251.7
195 245.4
0 0
199 25.4
198 237.29999999999998
198 222.0
198 206.7
198 107.0
0 0
198 252.6
198 251.5
197 187.0
197 155.2
197 211.6
198 253.9
0 0
199 232.0
199 125.5
199 106.89999999999999
199 209.10000000000002
199 189.3
199 176.7
199 252.6
0 0
199 253.9
198 235.4
197 252.9
196 249.6
198 50.8
191 251.6
199 203.2
199 177.29999999999998
199 142.20000000000002
199 116.10000000000001
194 225.9
193 227.70000000000002
191 249.8
191 250.9
0 0
199 209.9
199 250.0
199 253.9
191 252.9
195 204.0
199 28.4
193 228.6
0 0
0 0
195 250.4
0 0
191 254.0
195 25.5
199 216.9
196 227.70000000000002
196 152.1
186 128.70000000000002
194 200.0
184 243.3
184 252.0
184 252.9
0 0
0 0
0 0
197 61.199999999999996
199 227.70000000000002
195 237.5
195 252.8
196 227.70000000

In [26]:
print(decision_tree.testing_error(image_matrix[0:1200,],label_array[0:1200]))

0.095


In [11]:
decision_tree.train_model(image_matrix[0:1000,],label_array[0:1000],5)

TypeError: train_model() takes 3 positional arguments but 4 were given

In [ ]:
import numpy as np
import scipy.io as spio
import scipy.stats as stats
import math
import time
from sortedcollections import SortedSet

def build_step_for_feature(image_matrix):
    move_step = []

    for i in range(image_matrix.shape[1]):
        a = SortedSet(image_matrix[:,i].tolist())
        move_step.append(a)
    
    return move_step

class Node:
    def __init__(self,feature,threshold,left,right,label):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.label=label

def calculate_max_label_count(labels):
    count_map={}
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
            
     ##check this       
    return max(count_map.values())
    
def calculate_max_label(labels):
    count_map={}
    curr_max=0
    max_label=0
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
        if(count_map[labels[i]]>curr_max):
            curr_max=count_map[labels[i]]
            max_label=labels[i]
     ##check this       
    return max_label

def entropy(labels):
    if labels is None or len(labels)==0:
        return 0;
    return (len(labels)-calculate_max_label_count(labels))*1.0*(1.0/len(labels))
    

def calculate_optimal_feature_threshold(feature_space,labels):
    opt_f=0
    opt_t=0
    max_uncert_red=0.0
    
    for f in range(feature_space.shape[1]):
        #print(range(min(feature_space[:,f]),max(feature_space[:,f]),1))
        #for t in range(min(feature_space[:,f]),min(max(feature_space[:,f]),500),10):
        for t in move_step[f]:
            left_labels=[]
            right_labels=[]
            for i in range(len(feature_space)):
                if(feature_space[i,f]<t):
                    left_labels.append(labels[i])
                else:
                    right_labels.append(labels[i])
            if(len(left_labels)==0 or len(right_labels)==0):
                continue
            uncertainty_red = entropy(labels)-(len(left_labels)*1.0/len(labels))*entropy(left_labels) - (len(left_labels)*1.0/len(labels))*entropy(right_labels)
            if(uncertainty_red > max_uncert_red):
                opt_f=f
                opt_t=t
    return [opt_f,opt_t]

def build_tree(feature_space,labels,param,live_count):
    global tree_depth
    if len(feature_space)<=param or live_count > 9:
        return Node(None,None,None,None,calculate_max_label(labels))
    
    live_count += 1
    tree_depth = max(tree_depth,live_count)
    print tree_depth
    
    left_space=[]
    left_labels=[]
    right_space=[]
    right_labels=[]
    
    [f,t]=calculate_optimal_feature_threshold(feature_space,labels)
    print(f,t)
    
    for i in range(len(feature_space)):
        if(feature_space[i][f]<t):
            
            left_space.append(feature_space[i])
            left_labels.append(labels[i])
        else:
            right_space.append(feature_space[i])
            right_labels.append(labels[i])
    
    left_node=build_tree(np.array(left_space),left_labels,param,live_count)
    right_node=build_tree(np.array(right_space),right_labels,param,live_count)
    return Node(f,t,left_node,right_node,None)
    
    ## find the optimal feature and threshold
    ## split on the optimal feature and threshold
    ## node.left=build_tree(left_space)
    ## node.right=build_tree(right_space)
    ## return node


class DecisionTree:
    
    def __init__(self,num_classes,root):
        self.num_classes=num_classes
        self.root=root

                

    def train_model(self,training_data,labels):
        print("------Pre-processing Data-----")
        for i in range(len(training_data)):
            if (i == 0):
                feature_mle_mu = training_data[i]
                self.dim = (training_data[i].shape)[0]
            else:
                feature_mle_mu += training_data[i]
            
        feature_mle_mu = feature_mle_mu * 1.0/len(training_data)
        
        for i in range(len(training_data)):
            x_minus_mu_feature = training_data[i]-feature_mle_mu
            if (i == 0):
                feature_mle_sigma = x_minus_mu_feature**2
            else:
                feature_mle_sigma += x_minus_mu_feature**2
                
        feature_mle_sigma = feature_mle_sigma * 1.0/len(training_data)
        
        keys = np.linspace(0,self.dim-1,self.dim)
        feature_var = list(zip(keys,feature_mle_sigma))
        
        feature_var_sorted = sorted(feature_var, key=lambda f: f[1], reverse = True)
        self.top200 = [int(e[0]) for e in feature_var_sorted[:200]]  
        print("------Done Pre-processing Data -----")
        # slice the data
        training_data = training_data[:,self.top200]
        t0 = time.time()
        self.root=build_tree(training_data,labels,40,0)
        t1 = time.time()
        print ("Total time taken into building the tree: " + str(t1-t0))

    def predict(self,x):
        prediction=None
        curr_node=self.root
        while prediction==None:
            if curr_node.label!= None:
                prediction=curr_node.label
            else:
                if(x[curr_node.feature]<curr_node.threshold):
                    curr_node=curr_node.left
                else:
                    curr_node=curr_node.right
        return prediction
                        
    def testing_error(self,test_data,labels):
        misses=0
        test_data = test_data[:,self.top200]
        for i in range(len(test_data)):
            if(self.predict(test_data[i])!=labels[i]):
                misses=misses+1
        return (misses*1.0/len(test_data))
        ##Steps:
        ##1: return argmax P[X=x/Y=y]*P[Y=y]

#Partition = 0.7 for 70-30 split
def shuffle_and_split(data,partition):
    ran_order = np.arange(len(data['X']))
    np.random.shuffle(ran_order)
    ran_order_training = ran_order[:(int)(len(data['X'])*partition)] 
    ran_order_test = ran_order[(int)(len(data['X'])*partition):] 
    training_data = data['X'][ran_order_training]
    training_label = data['Y'][ran_order_training]
    test_data = data['X'][ran_order_test]
    test_label = data['Y'][ran_order_test]
    return [training_data,training_label,test_data,test_label]
        
mat = spio.loadmat('hw1data.mat', squeeze_me=True)
data_split = shuffle_and_split(mat,0.1)
image_matrix = data_split[0]
move_step = build_step_for_feature(image_matrix)
tree_depth = 0
label_array= data_split[1]

decision_tree= DecisionTree(10,None)
#[f,t]= calculate_optimal_feature_threshold(image_matrix[:30],label_array[:30])
decision_tree.train_model(image_matrix,label_array)

#Training error
print(decision_tree.testing_error(data_split[0],data_split[1]))
#Test Error
print(decision_tree.testing_error(data_split[2],data_split[3]))